In [85]:
import pandas as pd
import sqlite3
import numpy as np
pd.set_option('display.max_columns', None)

In [86]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/valorant.sqlite")
#game_rounds = pd.read_sql_query("SELECT * from Game_Rounds", con)
game_scoreboard = pd.read_sql_query("SELECT * from Game_Scoreboard", con)
games = pd.read_sql_query("SELECT * from games", con)
#matches = pd.read_sql_query("SELECT * from matches", con)

### Clean 'game_scoreboard'

In [87]:
game_scoreboard.head()

,GameID,PlayerID,PlayerName,TeamAbbreviation,Agent,ACS,Kills,Deaths,Assists,PlusMinus,KAST_Percent,ADR,HS_Percent,FirstKills,FirstDeaths,FKFD_PlusMinus,Num_2Ks,Num_3Ks,Num_4Ks,Num_5Ks,OnevOne,OnevTwo,OnevThree,OnevFour,OnevFive,Econ,Plants,Defuses
0,60894,8419,Reduxx,Boos,jett,313.0,24.0,10.0,3.0,14.0,0.65,195.0,0.31,4.0,4.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,74.0,0.0,0.0
1,60894,466,ChurmZ,Boos,chamber,227.0,16.0,10.0,7.0,6.0,0.90,161.0,0.16,1.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.0,2.0,0.0
2,60894,3712,diaamond,Boos,sova,226.0,17.0,9.0,8.0,8.0,0.85,148.0,0.27,3.0,0.0,3.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,58.0,3.0,0.0
3,60894,5099,Boltzy,Boos,viper,218.0,17.0,12.0,2.0,5.0,0.70,141.0,0.19,3.0,0.0,3.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,48.0,0.0,0.0
4,60894,3983,Virtyy,Boos,skye,80.0,5.0,13.0,3.0,-8.0,0.70,55.0,0.22,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0


#### Keep Teams with Games Played > 600

In [88]:
game_scoreboard_reduced = game_scoreboard[game_scoreboard.groupby('TeamAbbreviation')['TeamAbbreviation'].transform('size') > 600]

#### Remove Unnecessary Columns

In [89]:
columns_to_keep = ['GameID', 'TeamAbbreviation', 'ACS', 'ADR', 'HS_Percent', 'FirstKills', 'FirstDeaths'] # List of columns to keep
game_scoreboard_reduced = game_scoreboard_reduced[columns_to_keep]

#### Remove NaN Team Rows

In [90]:
game_scoreboard_reduced['TeamAbbreviation'].replace('', np.nan, inplace=True) # Replace '' with NaN
game_scoreboard_reduced.dropna(subset=['TeamAbbreviation'], inplace=True) # Remove all NaN

#### Caluclate Team Average ACS, ADR, HS_Percent, FirstKills, FirstDeaths

In [91]:
team_scoreboard = game_scoreboard_reduced.groupby(['TeamAbbreviation'])[['ACS', 'ADR', 'HS_Percent', 'FirstKills', 'FirstDeaths']].mean().reset_index()

#### Create 'GamesPlayed' Column

In [92]:
temp_df = game_scoreboard_reduced.groupby(['GameID','TeamAbbreviation'])[['ACS']].mean().reset_index()

In [93]:
games_played_map = temp_df['TeamAbbreviation'].value_counts().to_dict()

In [94]:
team_scoreboard['Games Played'] = team_scoreboard['TeamAbbreviation'].map(games_played_map)

In [95]:
team_scoreboard.head(10)

,TeamAbbreviation,ACS,ADR,HS_Percent,FirstKills,FirstDeaths,Games Played
0,100T,210.519185,136.751381,0.230083,2.111511,2.022099,168
1,ABX,214.374233,137.646626,0.264920,2.193865,1.904294,164
2,AE,203.372035,130.253769,0.252010,2.128589,2.066583,166
3,BBG,210.984946,134.413636,0.198000,2.084946,2.038596,186
4,BME,208.746939,133.449315,0.228476,2.236735,1.946853,147
5,C9,211.311195,136.310962,0.220246,2.168880,1.914989,215
6,COL,213.593909,139.006452,0.234967,2.167513,1.845055,197
7,EG,208.543732,137.384840,0.268192,2.119534,2.013120,140
8,FF,205.600000,134.555556,0.250825,2.103876,2.044444,129
9,FaZe,211.504348,135.306122,0.233565,2.183851,1.868027,161


### Clean 'games'

In [96]:
games.head()

,GameID,MatchID,Map,Team1ID,Team2ID,Team1,Team2,Winner,Team1_TotalRounds,Team2_TotalRounds,Team1_SideFirstHalf,Team2_SideFirstHalf,Team1_RoundsFirstHalf,Team1_RoundsSecondtHalf,Team1_RoundsOT,Team2_RoundsFirstHalf,Team2_RoundsSecondtHalf,Team2_RoundsOT,Team1_PistolWon,Team1_Eco,Team1_EcoWon,Team1_SemiEco,Team1_SemiEcoWon,Team1_SemiBuy,Team1_SemiBuyWon,Team1_FullBuy,Team1_FullBuyWon,Team2_PistolWon,Team2_Eco,Team2_EcoWon,Team2_SemiEco,Team2_SemiEcoWon,Team2_SemiBuy,Team2_SemiBuyWon,Team2_FullBuy,Team2_FullBuyWon
0,60894,62393,Breeze,6903,6020,Booster Seat Gaming,Pho Real,Booster Seat Gaming,13,7,defend,attack,7,6,0,5,2,0,2.0,2.0,2.0,0.0,0.0,5.0,3.0,13.0,8.0,0.0,4.0,0.0,2.0,0.0,4.0,1.0,10.0,6.0
1,60895,62393,Bind,6903,6020,Booster Seat Gaming,Pho Real,Pho Real,2,13,defend,attack,2,0,0,10,3,0,0.0,3.0,0.0,3.0,0.0,5.0,1.0,4.0,1.0,2.0,2.0,2.0,0.0,0.0,4.0,3.0,9.0,8.0
2,60896,62393,Haven,6903,6020,Booster Seat Gaming,Pho Real,Booster Seat Gaming,13,8,attack,defend,8,5,0,4,4,0,1.0,2.0,1.0,2.0,1.0,5.0,2.0,12.0,9.0,1.0,2.0,1.0,2.0,0.0,6.0,2.0,11.0,5.0
3,60924,62403,Icebox,7046,7047,Bjor's Kittens,Mugiwara,Bjor's Kittens,13,6,defend,attack,7,6,0,5,1,0,2.0,2.0,2.0,2.0,0.0,4.0,3.0,11.0,8.0,0.0,4.0,0.0,1.0,0.0,2.0,1.0,12.0,5.0
4,60925,62403,Haven,7046,7047,Bjor's Kittens,Mugiwara,Bjor's Kittens,13,9,defend,attack,9,4,0,3,6,0,1.0,4.0,2.0,2.0,0.0,3.0,0.0,13.0,11.0,1.0,3.0,2.0,3.0,0.0,4.0,3.0,12.0,4.0


In [97]:
# Split games into Team1 and Team2
games_team1 = games.filter(regex='Team1') 
games_team1['Win'] = np.where(games['Team1'] == games['Winner'], 1, 0)
games_team2 = games.filter(regex='Team2')
games_team2['Win'] = np.where(games['Team2'] == games['Winner'], 1, 0)

C:\Users\marco\AppData\Local\Temp\ipykernel_20640\2929630983.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_team1['Win'] = np.where(games['Team1'] == games['Winner'], 1, 0)
C:\Users\marco\AppData\Local\Temp\ipykernel_20640\2929630983.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_team2['Win'] = np.where(games['Team2'] == games['Winner'], 1, 0)


In [98]:
games_team2.columns = games_team1.columns # Match the column names
games_merged = pd.concat([games_team1, games_team2], ignore_index=True) # Concat the two dataframes
games_merged.columns = games_merged.columns.str.replace('1', '') # Clean column names of '1'

#### Create Team to TeamAbbreviation Mapping

No common columns exists in the dataframes, mapping dictionary must be done manually

In [99]:
team_mapping_dict = {
    '100 Thieves': '100T',
    'Andbox': 'ABX',
    'Alter Ego': 'AE',
    'Built By Gamers': 'BBG',
    'BOOM Esports': 'BME',
    'Cloud9': 'C9',
    'Complexity': 'COL',
    'Evil Geniuses': 'EG',
    'Fire Flux Esports': 'FF',
    'FaZe Clan': 'FaZe',
    'G2 Esports': 'G2',
    'Global Esports': 'GE',
    'Gen.G': 'GEN',
    'Gambit Esports': 'GMB',
    'Guild Esports': 'GUILD',
    'Ghost Gaming': 'Ghost',
    'Immortals': 'IMT',
    'Luminosity': 'LG',
    'Mindfreak': 'MF',
    'NRG Esports': 'NRG',
    'Envy': 'NV',
    'Knights': 'PK',
    'Paper Rex': 'PRX',
    'Royal Never Give Up': 'RNG',
    'Rise': 'Rise',
    'Sentinels': 'SEN',
    'Team SMG': 'SMG',
    'Soniqs': 'SQ',
    'T1': 'T1',
    'Team Basilisk': 'TB',
    'Team Liquid': 'TL',
    'Team Secret': 'TS',
    'TSM': 'TSM',
    'Team Reckaroo': 'Team',
    'Velocity Gaming': 'VLT',
    'XSET': 'XSET',
    'forZe': 'forZ'
    # Add more key-value pairs as needed
}

In [100]:
games_merged['Team'] = games_merged['Team'].map(team_mapping_dict) # Map with dictionary, values not in dictionary result with NaN
games_merged.dropna(subset=['Team'], inplace=True) # Drop rows with NaN values in 'Team'

#### Remove NaN Rows

In [101]:
nan_counts = games_merged.isna().sum()
print(nan_counts)

TeamID                      0
Team                        0
Team_TotalRounds            0
Team_SideFirstHalf          0
Team_RoundsFirstHalf        0
Team_RoundsSecondtHalf      0
Team_RoundsOT               0
Team_PistolWon            511
Team_Eco                  511
Team_EcoWon               511
Team_SemiEco              511
Team_SemiEcoWon           511
Team_SemiBuy              511
Team_SemiBuyWon           511
Team_FullBuy              511
Team_FullBuyWon           511
Win                         0
dtype: int64


In [102]:
games_merged.dropna(subset=['Team_PistolWon'], inplace=True) # Drop rows with NaN values in 'Team_PistolWon'

col_to_remove = ['TeamID', 'Team_SideFirstHalf'] # Columns to remove
games_merged = games_merged.drop(col_to_remove, axis=1) # Remove the columns

#### Create New Dataframe of Team Stats

In [103]:
col_names = games_merged.columns.tolist()[1:] # List excludes 'Team'
team_stats = games_merged.groupby(['Team'])[col_names].mean().reset_index()

#### Calculate 'x' Type Buy Win Conversion

In [104]:
# Calculate Buy Type Win Conversion
team_stats['Team_Eco_Conv'] = (team_stats['Team_EcoWon'] / team_stats['Team_Eco']) * 100
team_stats['Team_SemiEco_Conv'] = (team_stats['Team_SemiEcoWon'] / team_stats['Team_SemiEco']) * 100
team_stats['Team_SemiBuy_Conv'] = (team_stats['Team_SemiBuyWon'] / team_stats['Team_SemiBuy']) * 100
team_stats['Team_FullBuy_Conv'] = (team_stats['Team_FullBuyWon'] / team_stats['Team_FullBuy']) * 100
team_stats = team_stats.rename(columns={'Win': 'WinRate'}) # Rename 'Win' to 'WinRate '
team_stats['WinRate'] = team_stats['WinRate'] * 100
team_stats.rename(columns={'Team': 'TeamAbbreviation'}, inplace=True) 

In [105]:
team_stats = team_stats.drop(col_names[5:13], axis=1) # Drop 'x' Type Buy Columns

#### Create final dataset

In [108]:
data = pd.merge(team_scoreboard, team_stats, on='TeamAbbreviation')

In [109]:
data.head()

,TeamAbbreviation,ACS,ADR,HS_Percent,FirstKills,FirstDeaths,Games Played,Team_TotalRounds,Team_RoundsFirstHalf,Team_RoundsSecondtHalf,Team_RoundsOT,Team_PistolWon,WinRate,Team_Eco_Conv,Team_SemiEco_Conv,Team_SemiBuy_Conv,Team_FullBuy_Conv
0,100T,210.519185,136.751381,0.230083,2.111511,2.022099,168,11.312057,6.609929,4.489362,0.212766,1.063830,65.248227,39.401496,20.000000,58.364312,60.957746
1,ABX,214.374233,137.646626,0.264920,2.193865,1.904294,164,11.509202,6.981595,4.325153,0.202454,1.110429,63.803681,41.101695,20.276498,60.358891,62.217861
2,AE,203.372035,130.253769,0.252010,2.128589,2.066583,166,11.654867,6.637168,4.814159,0.203540,1.168142,64.601770,45.510836,22.413793,56.532067,60.583446
3,BBG,210.984946,134.413636,0.198000,2.084946,2.038596,186,11.366864,6.508876,4.745562,0.112426,1.130178,62.721893,44.175824,22.222222,57.412399,60.291262
4,BME,208.746939,133.449315,0.228476,2.236735,1.946853,147,11.419580,6.461538,4.741259,0.216783,1.118881,63.636364,43.002545,27.567568,54.409567,60.287356


#### Save data

In [84]:
#data.to_csv("valorant_team_data.csv", index=False)